# What are Agents? -or-
# LangChain Agents


In Langchain, an Agent is a decision-making component that uses LLM to decide what action to take next -- based on user input, context and available tools

Reasoning LLM  ---- LLM will decide from where to derive the solution

The goal of these kind of applications is to achieve AUTONOMY !!!
Let System Do THE DECISION-MAKING favorable for the business

# Key Jargon for Agents

1. LLM as a Reasoner: This LLM doesnt just answer. ---- IT PLANS. It decides which step or tool to use next.

2. Tools: External functions or API or MCP Servers that the agent can use (e.g. calculator, search, database query etc. )

3. Prompting Loop (ReAct): The agent uses the LLM repeatedly to REASON,

                    THINK -> ACT -> OBSERVE -> THINK -> ACT

4. Intermediate Steps: Logs of all tool invocations and their outputs used to refine the reasoning.

5. Output Parsers: Defines how the LLMs text output is converted into STRUCTURED ACTIONS or RESPONSES.

# Agent's Thought Process | Agent's Thought Cycle


A typical agent operation may look like below:

1. Receive user query (What is the current temperature in Pune and how is it as compared to yesterday?)

2. Reasoning Step (LLM decide what to do:)
 -> I need to use a Weather API tool to get today's weather and get yesterday's weather

3. Action Step
 -> Agent calls get_weather(city="Pune")

4. Observation
 -> The tool returned data -> Agent update its context

5. Next Reasoning Step:
 -> LLM decides to compare both day's data.

6. Final Answer:
 -> Agent generates a Natural Language Reply to the user.

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents.openai_functions_agent.base import create_openai_functions_agent
from langchain.agents import AgentExecutor

In [3]:
# 1) Define a tool
@tool
def get_weather(location: str) -> str:
    """Return a fake weather forecast for the given location."""
    # In a real build, you’d call a weather API here
    return f"The weather in {location} is sunny and 25 °C."

tools = [get_weather]

In [4]:
# 2) Set up the LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

In [5]:
# 3) Build the agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can call a weather tool."),
    MessagesPlaceholder("agent_scratchpad"),  #Agent Scratchpad maintains all intermediate outputs
    ("human", "{input}")
])

#Initializing the agent
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)

#Making the agent executable | query-able
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [6]:
# 4) Invoke the agent
result = agent_executor.invoke({"input": "What’s the weather in Pune?"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'location': 'Pune'}`


The weather in Pune is sunny and 25 °C.The weather in Pune is sunny with a temperature of 25 °C.

> Finished chain.
{'input': 'What’s the weather in Pune?', 'output': 'The weather in Pune is sunny with a temperature of 25 °C.'}


In [7]:
# 4) Invoke the agent
result = agent_executor.invoke({"input": "What’s 2+2?"})
print(result)



> Entering new AgentExecutor chain...
2 + 2 equals 4.

> Finished chain.
{'input': 'What’s 2+2?', 'output': '2 + 2 equals 4.'}


In [9]:
from langchain.agents import create_tool_calling_agent
from langchain_google_genai import ChatGoogleGenerativeAI

llmGemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash"
)

agent2 = create_tool_calling_agent(
    llm=llmGemini,
    tools=tools,
    prompt=prompt
) 

executorGeminiAgent = AgentExecutor(agent=agent2, tools=tools, verbose=True)

executorGeminiAgent.invoke({"input": "Whats the weather in Pune?"})



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'location': 'Pune'}`


The weather in Pune is sunny and 25 °C.The weather in Pune is sunny and 25 °C.

> Finished chain.


{'input': 'Whats the weather in Pune?',
 'output': 'The weather in Pune is sunny and 25 °C.'}